In [3]:
## Attaches burn probabilities to the ZTRAX data
## Uses lat and long provided 
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import glob
import matplotlib.pyplot as plt
import rasterio as rio
from rasterio.plot import show
from functools import reduce

In [4]:
pd.options.display.max_columns = None


In [5]:
## Get US Map for context and select Western United States
##Convert Coordinate Reference System to US Natinoal Atlas Equal Area
USMAP = gpd.read_file('/data/yoder/DensityProject/cb_2018_us_state_500k.shp')
USMAP = USMAP.to_crs("EPSG:2163")
West = ['WA', 'OR', 'CA', 'NV','AZ','NM', 'UT','CO','WY','ID','MT']

## Get map(s) of individual states if desired
WestStates = USMAP[USMAP['STUSPS'].isin(West)]

In [6]:
scratchpath = '/data/yoder/AllZillow/HedonicData/'
os.chdir(scratchpath)
## List all tables included in Zillow Variable names excel file
ZTransLayout = pd.read_excel('Layout.xlsx','ZTrans')
ZTransLayout['TableName'].unique()
ZAsmtLayout = pd.read_excel('Layout.xlsx','ZAsmt')

## Create dataframes from each table name in the Zillow table dictionary
    ## Strip 'ut' from column names
T = {}
for name,group in ZTransLayout.groupby('TableName'):
    T[str(name)[2:]] = group
A = {}
for name, group in ZAsmtLayout.groupby('TableName'):
    A[str(name)[2:]] = group
    
    
## Dictionary of Western 11 states using Zillows numbering: 
states = {'CO':'08'}
# states = {'AZ':'04',
#           'CA':'06',
#           'CO':'08',
#           'ID':'16',
#           'MT':'30',
#           'NV':'32',
#           'NM':'35',
#           'OR':'41',
#           'UT':'49',
#           'WA':'53',
#           'WY':'56'}

In [7]:
#Change this value to the state and table you care about
# State to use
# state = states[West]
state_map = USMAP[USMAP['STUSPS']=='CO']
#Transaction tables to use
Ttables = ['Main', 'PropertyInfo']
# Assessment tables to use
Atables = ['Main','Garage', 'Building', 'Value']


## Fields to use from transaction tables
Tfields = {'Main': ['TransId','State','County','DocumentDate',
                    'RecordingDate','FIPS','DataClassStndCode',
                    'DocumentTypeStndCode',
                    'LoanTypeStndCode','PropertyUseStndCode',
                    'SalesPriceAmount', 'LoanAmount'],
            'PropertyInfo':['TransId','ImportParcelID','AssessorParcelNumber',
                            'PropertyFullStreetAddress','PropertyCity',
                            'PropertyState','PropertyAddressLatitude',
                            'PropertyAddressLongitude']}
## Fields not currently used
## 'IntraFamilyTransferFlag', 'SignatureDate',
# Fields to use from Assessment tables
Afields = {'Main': ['RowID','ImportParcelID','LotSizeSquareFeet'],
          'Garage':['RowID','GarageStndCode','GarageAreaSqFt'],
          'Building':['RowID','ArchitecturalStyleStndCode','BuildingClassStndCode','BuildingQualityStndCode',
                       'BuildingConditionStndCode','EffectiveYearBuilt','YearBuilt','YearRemodeled',
                       'NoOfStories','TotalRooms','TotalBedrooms','TotalKitchens',
                       'FullBath','ThreeQuarterBath','HalfBath','QuarterBath','TotalActualBathCount',
                       'TotalBathPlumbingFixtures','RoofCoverStndCode', 'RoofStructureTypeStndCode',
                       'HeatingTypeorSystemStndCode','AirConditioningTypeorSystemStndCode',
                       'FoundationTypeStndCode','ElevatorStndCode','FireplaceFlag',
                       'FireplaceNumber','WaterStndCode','SewerStndCode'],
          'Value': ['RowID','LandAssessedValue','ImprovementAssessedValue','TotalAssessedValue',
                    'AssessmentYear','LandMarketValue','ImprovementMarketValue','TotalMarketValue',
                    'MarketValueYear','LandAppraisalValue','ImprovementAppraisalValue','TotalAppraisalValue',
                    'AppraisalValueYear']} 
## Throws error if these are included 'FireplaceTypeStndCode','StoryTypeCode'
## Not currenlty used ,'TimeshareStndCode'

In [6]:
%%time
sts_trans = {}
sts_vals = {}
SetofStates = {}
for state in states.values():
    keys = Ttables
    dfs = {}
    for i in keys:
        print(f'Started loading {state} table {i}')
        dfs[i] = pd.read_csv(f'{state}/ZTrans\{i}.txt',sep = '|', header=None,names=T[i]['FieldName'].tolist(), encoding='latin1',usecols=Tfields[i],low_memory=False)
        print(f'Finished loading {state} Trans, table {i}')
    sts_trans[state] = reduce(lambda left,right: pd.merge(left,right,on='TransId',how='left'),dfs.values())
    print(f'Finished All Transactions Tables')
    dfA = {}
    keys = Atables
    for j in keys:
        print(f'Started Loading {state} Asmt, table {j} ')
        dfA[j] = pd.read_csv(f'{state}/ZAsmt\{j}.txt',sep='|', header=None,names=A[j]['FieldName'].tolist(),encoding='latin1',usecols=Afields[j],low_memory=False)
        print(f'Finished Loading {state} Asmt, table {j}')
    sts_vals[state] =  reduce(lambda left, right:pd.merge(left,right,on='RowID',how='left'),dfA.values())


Started loading 08 table Main
Finished loading 08 Trans, table Main
Started loading 08 table PropertyInfo
Finished loading 08 Trans, table PropertyInfo
Finished All Transactions Tables
Started Loading 08 Asmt, table Main 
Finished Loading 08 Asmt, table Main
Started Loading 08 Asmt, table Garage 
Finished Loading 08 Asmt, table Garage
Started Loading 08 Asmt, table Building 
Finished Loading 08 Asmt, table Building
Started Loading 08 Asmt, table Value 
Finished Loading 08 Asmt, table Value
CPU times: user 2min 20s, sys: 52.8 s, total: 3min 13s
Wall time: 3min 29s


In [7]:
SetofStates[state] = pd.merge(sts_trans[state],sts_vals[state],on='ImportParcelID', how='left')

In [12]:
sts_vals[state]

,RowID,ImportParcelID,LotSizeSquareFeet,GarageStndCode,GarageAreaSqFt,BuildingClassStndCode,BuildingQualityStndCode,BuildingConditionStndCode,ArchitecturalStyleStndCode,YearBuilt,EffectiveYearBuilt,YearRemodeled,NoOfStories,TotalRooms,TotalBedrooms,TotalKitchens,FullBath,ThreeQuarterBath,HalfBath,QuarterBath,TotalActualBathCount,TotalBathPlumbingFixtures,RoofCoverStndCode,RoofStructureTypeStndCode,HeatingTypeorSystemStndCode,AirConditioningTypeorSystemStndCode,FoundationTypeStndCode,ElevatorStndCode,FireplaceFlag,FireplaceNumber,WaterStndCode,SewerStndCode,LandAssessedValue,ImprovementAssessedValue,TotalAssessedValue,AssessmentYear,LandMarketValue,ImprovementMarketValue,TotalMarketValue,MarketValueYear,LandAppraisalValue,ImprovementAppraisalValue,TotalAppraisalValue,AppraisalValueYear
0,CB00D515-820C-E811-80C3-3863BB430E3F,21607550,10019.0,AT,NaN,NaN,C,AV,NaN,2002.0,NaN,NaN,2.0,9.0,4.0,NaN,NaN,NaN,NaN,NaN,2.5,13.0,CS,GBL,FA,NaN,NaN,NaN,NaN,1.0,NaN,NaN,6120.0,21700.0,27820.0,2017,85000.0,301337.0,386337.0,2017.0,NaN,NaN,NaN,NaN
1,CB00D515-820C-E811-80C3-3863BB430E3F,21607550,10019.0,GR,1.0,NaN,C,AV,NaN,2002.0,NaN,NaN,2.0,9.0,4.0,NaN,NaN,NaN,NaN,NaN,2.5,13.0,CS,GBL,FA,NaN,NaN,NaN,NaN,1.0,NaN,NaN,6120.0,21700.0,27820.0,2017,85000.0,301337.0,386337.0,2017.0,NaN,NaN,NaN,NaN
2,CC00D515-820C-E811-80C3-3863BB430E3F,21684561,6534.0,DT,NaN,NaN,C,AV,RA,1949.0,NaN,NaN,1.0,4.0,2.0,NaN,1.0,NaN,0.0,NaN,NaN,6.0,CS,GBL,FA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4320.0,9400.0,13720.0,2017,60000.0,130622.0,190622.0,2017.0,NaN,NaN,NaN,NaN
3,CC00D515-820C-E811-80C3-3863BB430E3F,21684561,6534.0,GR,1.0,NaN,C,AV,RA,1949.0,NaN,NaN,1.0,4.0,2.0,NaN,1.0,NaN,0.0,NaN,NaN,6.0,CS,GBL,FA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4320.0,9400.0,13720.0,2017,60000.0,130622.0,190622.0,2017.0,NaN,NaN,NaN,NaN
4,CD00D515-820C-E811-80C3-3863BB430E3F,21577693,7841.0,AT,NaN,NaN,B,AV,NaN,2001.0,NaN,NaN,2.0,8.0,4.0,NaN,3.0,NaN,0.0,NaN,NaN,13.0,CS,GBL,NaN,CE,NaN,NaN,NaN,1.0,NaN,NaN,7490.0,19060.0,26550.0,2017,104000.0,264786.0,368786.0,2017.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3971918,8C032423-6A47-E811-80C3-3863BB430E3F,21774073,6926.0,AT,NaN,NaN,C,AV,NaN,1980.0,NaN,NaN,NaN,0.0,2.0,NaN,2.0,NaN,0.0,NaN,NaN,6.0,CS,GHP,FA,EC,CN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,19030.0,2017,55000.0,209300.0,264300.0,2017.0,NaN,NaN,NaN,NaN
3971919,8C032423-6A47-E811-80C3-3863BB430E3F,21774073,6926.0,GR,440.0,NaN,C,AV,NaN,1980.0,NaN,NaN,NaN,0.0,2.0,NaN,2.0,NaN,0.0,NaN,NaN,6.0,CS,GHP,FA,EC,CN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,19030.0,2017,55000.0,209300.0,264300.0,2017.0,NaN,NaN,NaN,NaN
3971920,8D032423-6A47-E811-80C3-3863BB430E3F,21911571,9453.0,AT,NaN,NaN,C,AV,RA,1950.0,NaN,NaN,1.0,0.0,0.0,NaN,1.0,NaN,0.0,NaN,NaN,3.0,CS,GHP,FA,NO,CN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22824.0,2017,137500.0,179500.0,317000.0,2017.0,NaN,NaN,NaN,NaN
3971921,8D032423-6A47-E811-80C3-3863BB430E3F,21911571,9453.0,GR,308.0,NaN,C,AV,RA,1950.0,NaN,NaN,1.0,0.0,0.0,NaN,1.0,NaN,0.0,NaN,NaN,3.0,CS,GHP,FA,NO,CN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22824.0,2017,137500.0,179500.0,317000.0,2017.0,NaN,NaN,NaN,NaN


In [13]:
%%time
df = pd.concat(SetofStates.values(), ignore_index=True)

CPU times: user 2.6 s, sys: 4.14 s, total: 6.74 s
Wall time: 6.74 s


In [14]:
df

,TransId,FIPS,State,County,DataClassStndCode,RecordingDate,DocumentTypeStndCode,DocumentDate,SalesPriceAmount,PropertyUseStndCode,LoanAmount,LoanTypeStndCode,AssessorParcelNumber,PropertyFullStreetAddress,PropertyCity,PropertyState,PropertyAddressLatitude,PropertyAddressLongitude,ImportParcelID,RowID,LotSizeSquareFeet,GarageStndCode,GarageAreaSqFt,BuildingClassStndCode,BuildingQualityStndCode,BuildingConditionStndCode,ArchitecturalStyleStndCode,YearBuilt,EffectiveYearBuilt,YearRemodeled,NoOfStories,TotalRooms,TotalBedrooms,TotalKitchens,FullBath,ThreeQuarterBath,HalfBath,QuarterBath,TotalActualBathCount,TotalBathPlumbingFixtures,RoofCoverStndCode,RoofStructureTypeStndCode,HeatingTypeorSystemStndCode,AirConditioningTypeorSystemStndCode,FoundationTypeStndCode,ElevatorStndCode,FireplaceFlag,FireplaceNumber,WaterStndCode,SewerStndCode,LandAssessedValue,ImprovementAssessedValue,TotalAssessedValue,AssessmentYear,LandMarketValue,ImprovementMarketValue,TotalMarketValue,MarketValueYear,LandAppraisalValue,ImprovementAppraisalValue,TotalAppraisalValue,AppraisalValueYear
0,97922611,8001,CO,ADAMS,H,1993-09-01,WRDE,NaN,75000.0,NaN,75900.0,NaN,1719-14-4-02-039,1860 E 98TH PL,THORNTON,CO,39.876425,-104.964230,21608543.0,42751A16-820C-E811-80C3-3863BB430E3F,7841.0,AT,NaN,NaN,C,AV,RA,1975.0,NaN,NaN,1.0,5.0,3.0,NaN,1.0,NaN,0.0,NaN,NaN,896.0,CS,GBL,FA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5760.0,10060.0,15820.0,2017.0,80000.0,139740.0,219740.0,2017.0,NaN,NaN,NaN,NaN
1,97922611,8001,CO,ADAMS,H,1993-09-01,WRDE,NaN,75000.0,NaN,75900.0,NaN,1719-14-4-02-039,1860 E 98TH PL,THORNTON,CO,39.876425,-104.964230,21608543.0,42751A16-820C-E811-80C3-3863BB430E3F,7841.0,GR,816.0,NaN,C,AV,RA,1975.0,NaN,NaN,1.0,5.0,3.0,NaN,1.0,NaN,0.0,NaN,NaN,896.0,CS,GBL,FA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5760.0,10060.0,15820.0,2017.0,80000.0,139740.0,219740.0,2017.0,NaN,NaN,NaN,NaN
2,97922612,8001,CO,ADAMS,H,1993-09-01,WRDE,NaN,194100.0,UL,120000.0,NaN,1719-05-1-10-031,11673 DECATUR DR,WESTMINSTER,CO,39.908376,-105.019622,21593845.0,19ED3B16-820C-E811-80C3-3863BB430E3F,7841.0,AT,NaN,NaN,B,GD,NaN,1993.0,2013.0,2013.0,2.0,7.0,5.0,NaN,NaN,NaN,NaN,NaN,2.5,15.0,WS,GBL,FA,NaN,NaN,NaN,NaN,1.0,NaN,NaN,9360.0,19730.0,29090.0,2017.0,130000.0,274070.0,404070.0,2017.0,NaN,NaN,NaN,NaN
3,97922612,8001,CO,ADAMS,H,1993-09-01,WRDE,NaN,194100.0,UL,120000.0,NaN,1719-05-1-10-031,11673 DECATUR DR,WESTMINSTER,CO,39.908376,-105.019622,21593845.0,19ED3B16-820C-E811-80C3-3863BB430E3F,7841.0,GR,1.0,NaN,B,GD,NaN,1993.0,2013.0,2013.0,2.0,7.0,5.0,NaN,NaN,NaN,NaN,NaN,2.5,15.0,WS,GBL,FA,NaN,NaN,NaN,NaN,1.0,NaN,NaN,9360.0,19730.0,29090.0,2017.0,130000.0,274070.0,404070.0,2017.0,NaN,NaN,NaN,NaN
4,97922613,8001,CO,ADAMS,H,1993-09-01,WRDE,NaN,114900.0,NaN,103410.0,NaN,1719-05-3-09-042,3265 W 115TH PL,WESTMINSTER,CO,39.905624,-105.027946,21594200.0,DA2E5D16-820C-E811-80C3-3863BB430E3F,5663.0,AT,NaN,NaN,B,AV,NaN,1990.0,NaN,NaN,NaN,5.0,3.0,NaN,2.0,NaN,0.0,NaN,NaN,9.0,CS,GBL,FA,NaN,NaN,NaN,NaN,1.0,NaN,NaN,9360.0,12700.0,22060.0,2017.0,130000.0,176455.0,306455.0,2017.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19877934,449200102,8125,CO,YUMA,D,2018-10-18,INTR,2000-06-22,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19877935,449200103,8125,CO,YUMA,D,2018-10-18,INTR,2000-01-21,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19877936,449200104,8125,CO,YUMA,D,2018-10-18,INTR,2000-06-27,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [15]:
## Select only valid transactions
UnwantedLoanCodes =  ['AC','CT','CM','CS','CC','CL','DP','FO','FE','HE','LC','EB','EX','MD','NA','NP','FA','RE','RM','SM','SE','TR']
WantedLoanCodes =  ['AS','BL','CE','FM','PM','RD','SL']
DocumentTypeStndCodeDrop= ['CRDE','JTDE','QCDE','RRDE','VLDE']
DocumentTypeStndCodeKeep= ['CPDE','IDDE','PRDE','WRDE']

In [16]:
df.columns

Index(['TransId', 'FIPS', 'State', 'County', 'DataClassStndCode',
       'RecordingDate', 'DocumentTypeStndCode', 'DocumentDate',
       'SalesPriceAmount', 'PropertyUseStndCode', 'LoanAmount',
       'LoanTypeStndCode', 'AssessorParcelNumber', 'PropertyFullStreetAddress',
       'PropertyCity', 'PropertyState', 'PropertyAddressLatitude',
       'PropertyAddressLongitude', 'ImportParcelID', 'RowID',
       'LotSizeSquareFeet', 'GarageStndCode', 'GarageAreaSqFt',
       'BuildingClassStndCode', 'BuildingQualityStndCode',
       'BuildingConditionStndCode', 'ArchitecturalStyleStndCode', 'YearBuilt',
       'EffectiveYearBuilt', 'YearRemodeled', 'NoOfStories', 'TotalRooms',
       'TotalBedrooms', 'TotalKitchens', 'FullBath', 'ThreeQuarterBath',
       'HalfBath', 'QuarterBath', 'TotalActualBathCount',
       'TotalBathPlumbingFixtures', 'RoofCoverStndCode',
       'RoofStructureTypeStndCode', 'HeatingTypeorSystemStndCode',
       'AirConditioningTypeorSystemStndCode', 'FoundationTypeStnd

In [17]:
## Select only valid transactions
st = df.loc[(df['SalesPriceAmount'] > 0) &
            (~df['LoanTypeStndCode'].isin(UnwantedLoanCodes)) &
#             (pd.isnull(df['IntraFamilyTransferFlag'])) &
            (df['PropertyUseStndCode']=='SR') &
            (df['DocumentTypeStndCode'].isin(DocumentTypeStndCodeKeep)) & 
            (df['TotalBedrooms'] >=1)].copy()

In [18]:
st = gpd.GeoDataFrame(st, geometry = gpd.points_from_xy(st.PropertyAddressLongitude, st.PropertyAddressLatitude))
st_proj = st.copy()
st_proj.crs="epsg:4326"
st_proj=st_proj.to_crs("EPSG:2163")
st_proj.crs

<Projected CRS: EPSG:2163>
Name: US National Atlas Equal Area
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: USA
- bounds: (167.65, 15.56, -65.69, 74.71)
Coordinate Operation:
- name: US National Atlas Equal Area
- method: Lambert Azimuthal Equal Area (Spherical)
Datum: Not specified (based on Clarke 1866 Authalic Sphere)
- Ellipsoid: Clarke 1866 Authalic Sphere
- Prime Meridian: Greenwich

In [19]:
import rasterio
from rasterstats import zonal_stats, point_query
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [20]:
## Reproject raster and load it

dst_crs ="EPSG:2163"

with rasterio.open('/data/yoder/Spread_risk/Spread_risk_raster/BP_2016raster.tif') as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })
    with rasterio.open('/data/yoder/Spread_risk/Spread_risk_raster/BP_2016raster_2163.tif', 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
                   reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)
                
src = rasterio.open('/data/yoder/Spread_risk/Spread_risk_raster/BP_2016raster_2163.tif')

In [21]:
## Plot points
fig, ax = plt.subplots()
show(src,ax = ax)
st_proj.plot(ax=ax, color ='red')

/home/joshua.olsen/.conda/envs/testenv/lib/python3.8/site-packages/matplotlib/image.py:490: RuntimeWarning: overflow encountered in true_divide
  A_scaled /= ((a_max - a_min) / frac)


<AxesSubplot:>

In [22]:
pts = st_proj.copy()

In [23]:
st_proj.shape

(1375077, 63)

In [24]:
## Select only valid geometry (note .isna() does not appear to catch point (nan,nan))
st_pts = st_proj[st_proj['geometry'].is_valid]
st_pts = gpd.clip(st_pts, state_map)

In [25]:
st_pts.shape

(1374190, 63)

In [26]:
st_pts.plot()

<AxesSubplot:>

In [27]:
pts = st_pts.copy()

In [28]:
%%time
## Add burn probability that the location is within (landfire uses 270 meter pixels)
pts['BP'] = point_query(pts, '/data/yoder/Spread_risk/Spread_risk_raster/BP_2016raster_2163.tif')

CPU times: user 49min 54s, sys: 3min 25s, total: 53min 19s
Wall time: 1h 1min 23s


In [29]:
%%time
## Add burn probability out to 1600 meters
pts['point_geometry'] = pts['geometry'].copy()
pts['geometry'] = pts.buffer(1600)
buff = pd.DataFrame(zonal_stats(pts, 
                             '/data/yoder/Spread_risk/Spread_risk_raster/BP_2016raster_2163.tif',
                            stats=['max','mean']))
buff.columns = [f'BP1600_'+str(col) for col in buff.columns]

buff.reset_index(drop=True,inplace=True)
pts.reset_index(drop=True,inplace=True)
pts = pd.concat([pts,buff],axis=1)
pts['geometry'] = pts['point_geometry'].copy()

CPU times: user 1h 16min 22s, sys: 7min 10s, total: 1h 23min 33s
Wall time: 1h 37min 47s


In [30]:
BurnCols = ['BP','BP1600_max','BP1600_mean']

In [32]:
%%time
ST_points = df.copy()
## Add fire perimeter data
import geopandas as gpd
Fireperims = gpd.read_file('/data/yoder/DensityProject/mtbs_perimeter_data/mtbs_perims_DD.shp')
Fireperims = Fireperims.to_crs("EPSG:2163")

state_map = state_map[['STUSPS', 'geometry']]
Buffed_state = state_map.copy()
Buffed_state['geometry'] = Buffed_state.buffer(20000)
ST_fires = gpd.sjoin(Fireperims, Buffed_state, op='within')
ST_fires = ST_fires.drop(['index_right'], axis=1)
ST_fires['Buffed_geometry100km'] = ST_fires.buffer(20000)
ST_fires['MTBS_geometry'] = ST_fires['geometry'].copy()
ST_fires['geometry'] = ST_fires['Buffed_geometry100km'].copy()


CPU times: user 27.2 s, sys: 599 ms, total: 27.8 s
Wall time: 28.1 s


In [31]:
df = pts.copy()

In [33]:
basemap = state_map.plot(edgecolor='black',color='white')
ST_fires.plot(ax=basemap, color='red')
ST_points.plot(ax= basemap, color='blue')

<AxesSubplot:>

In [34]:
%%time
ST_Samp_with_Fires = gpd.sjoin(ST_points, ST_fires, how='left')

CPU times: user 2min 38s, sys: 16.6 s, total: 2min 54s
Wall time: 2min 54s


In [35]:
def get_distance(row):
    distance = row.geometry.distance(row.MTBS_geometry)
    return distance

In [36]:
HomesAroundFires = ST_Samp_with_Fires[ST_Samp_with_Fires.MTBS_geometry.notnull()].copy()

In [37]:
HomesNotAroundFires = ST_Samp_with_Fires[ST_Samp_with_Fires.MTBS_geometry.isnull()].copy()

In [38]:
%%time
HomesAroundFires['Distance'] = HomesAroundFires.apply(lambda row: get_distance(row),axis=1)

CPU times: user 3min 3s, sys: 5.04 s, total: 3min 8s
Wall time: 3min 8s


In [39]:
import pickle
pickle.dump(HomesAroundFires, open( "COHomesAroundFires.p", "wb" ) )

In [1]:
import pickle

In [8]:
%%time
HomesAroundFires = pd.read_pickle('COHomesAroundFires.p')

CPU times: user 17.4 s, sys: 1.9 s, total: 19.3 s
Wall time: 26.2 s


In [40]:
df.shape

(1374190, 67)

In [10]:
HAF = HomesAroundFires

In [42]:
HNAF = HomesNotAroundFires

In [11]:
HAF

,TransId,FIPS,State,County,DataClassStndCode,RecordingDate,DocumentTypeStndCode,DocumentDate,SalesPriceAmount,PropertyUseStndCode,LoanAmount,LoanTypeStndCode,AssessorParcelNumber,PropertyFullStreetAddress,PropertyCity,PropertyState,PropertyAddressLatitude,PropertyAddressLongitude,ImportParcelID,RowID,LotSizeSquareFeet,GarageStndCode,GarageAreaSqFt,BuildingClassStndCode,BuildingQualityStndCode,BuildingConditionStndCode,ArchitecturalStyleStndCode,YearBuilt,EffectiveYearBuilt,YearRemodeled,NoOfStories,TotalRooms,TotalBedrooms,TotalKitchens,FullBath,ThreeQuarterBath,HalfBath,QuarterBath,TotalActualBathCount,TotalBathPlumbingFixtures,RoofCoverStndCode,RoofStructureTypeStndCode,HeatingTypeorSystemStndCode,AirConditioningTypeorSystemStndCode,FoundationTypeStndCode,ElevatorStndCode,FireplaceFlag,FireplaceNumber,WaterStndCode,SewerStndCode,LandAssessedValue,ImprovementAssessedValue,TotalAssessedValue,AssessmentYear,LandMarketValue,ImprovementMarketValue,TotalMarketValue,MarketValueYear,LandAppraisalValue,ImprovementAppraisalValue,TotalAppraisalValue,AppraisalValueYear,geometry,BP,point_geometry,BP1600_max,BP1600_mean,index_right,Fire_ID,Fire_Name,Year,StartMonth,StartDay,Fire_Type,Acres,STUSPS,Buffed_geometry100km,MTBS_geometry,Distance
40,97923044,8001,CO,ADAMS,H,1993-09-10,WRDE,NaN,64900.0,SR,65711.0,NaN,1825-18-1-14-001,5290 QUITMAN ST,DENVER,CO,39.792800,-105.039560,21694020.0,E2A75316-820C-E811-80C3-3863BB430E3F,7841.0,NaN,NaN,NaN,C,NaN,RA,1917.0,2001.0,2001.0,1.0,4.0,2.0,NaN,1.0,NaN,0.0,NaN,NaN,304.0,CS,GBL,NaN,CE,NaN,NaN,NaN,NaN,NaN,NaN,6480.0,12170.0,18650.0,2017.0,90000.0,168983.0,258983.0,2017.0,NaN,NaN,NaN,NaN,POINT (-430686.662 -565717.586),0.000098,POINT (-430686.662 -565717.586),0.00035,0.000034,4535.0,CO3976210526320110320,INDIANGULCHFIRE,2011.0,3.0,20.0,Wildfire,1640.0,CO,"POLYGON ((-471930.698 -570157.810, -472068.900...","POLYGON ((-451536.725 -569134.815, -451547.548...",18050.275697
64,97923703,8001,CO,ADAMS,H,1993-10-01,WRDE,NaN,66900.0,SR,67248.0,NaN,1719-31-4-12-041,7320 ORCHARD CT,WESTMINSTER,CO,39.829613,-105.037318,21635035.0,A225D515-820C-E811-80C3-3863BB430E3F,7405.0,NaN,NaN,NaN,C,AV,RA,1949.0,1980.0,1980.0,1.0,5.0,3.0,NaN,1.0,NaN,0.0,NaN,NaN,6.0,CS,GBL,NaN,CE,NaN,NaN,NaN,NaN,NaN,NaN,6120.0,8700.0,14820.0,2017.0,85000.0,120851.0,205851.0,2017.0,NaN,NaN,NaN,NaN,POINT (-430258.478 -561645.060),0.000000,POINT (-430258.478 -561645.060),0.00005,0.000003,4535.0,CO3976210526320110320,INDIANGULCHFIRE,2011.0,3.0,20.0,Wildfire,1640.0,CO,"POLYGON ((-471930.698 -570157.810, -472068.900...","POLYGON ((-451536.725 -569134.815, -451547.548...",19271.480956
66,97923833,8001,CO,ADAMS,H,1993-10-05,WRDE,NaN,65000.0,SR,65559.0,NaN,1825-17-2-11-010,5333 JULIAN ST,DENVER,CO,39.793609,-105.031472,21693738.0,F1034316-820C-E811-80C3-3863BB430E3F,12632.0,DT,NaN,NaN,C,GD,RA,1951.0,NaN,NaN,1.0,4.0,2.0,NaN,2.0,NaN,0.0,NaN,NaN,8.0,CS,GBL,NaN,CE,NaN,NaN,NaN,NaN,NaN,NaN,6480.0,17510.0,23990.0,2017.0,90000.0,243239.0,333239.0,2017.0,NaN,NaN,NaN,NaN,POINT (-429991.308 -565669.833),0.000000,POINT (-429991.308 -565669.833),0.00035,0.000032,4535.0,CO3976210526320110320,INDIANGULCHFIRE,2011.0,3.0,20.0,Wildfire,1640.0,CO,"POLYGON ((-471930.698 -570157.810, -472068.900...","POLYGON ((-451536.725 -569134.815, -451547.548...",18747.093695
67,97923833,8001,CO,ADAMS,H,1993-10-05,WRDE,NaN,65000.0,SR,65559.0,NaN,1825-17-2-11-010,5333 JULIAN ST,DENVER,CO,39.793609,-105.031472,21693738.0,F1034316-820C-E811-80C3-3863BB430E3F,12632.0,GR,1.0,NaN,C,GD,RA,1951.0,NaN,NaN,1.0,4.0,2.0,NaN,2.0,NaN,0.0,NaN,NaN,8.0,CS,GBL,NaN,CE,NaN,NaN,NaN,NaN,NaN,NaN,6480.0,17510.0,23990.0,2017.0,90000.0,243239.0,333239.0,2017.0,NaN,NaN,NaN,NaN,POINT (-429991.308 -565669.833),0.000000,POINT (-429991.308 -565669.833),0.00035,0.000032,4535.0,CO3976210526320110320,INDIANGULCHFIRE,2011.0,3.0,20.0,Wildfire,1640.0,CO,"POLYGON ((-471930.698 -570157.810, -472068.900...","POLYGON ((-451536.725 -569134.815, -451547.548...",18747.093695
74,97924030,8001,CO,ADAMS,H,1993-10-12,WRDE,NaN,82500.0

In [12]:
df = HAF

In [13]:
df['RecordingDate'] =pd.to_datetime(df['RecordingDate'])
df['Year'] = df.RecordingDate.dt.year

In [14]:
# HAF.groupby('BuildingQualityStndCode')['TransId'].count()

In [15]:
df.groupby('RoofCoverStndCode')['TransId'].count()

RoofCoverStndCode
AP     212455
AS       1221
BR        109
BU      26141
CN       6584
CS    1860090
FG         51
GR         44
ME      27003
OT       1045
RC       2199
SH       3710
SL        912
ST          4
TG        652
TL      48841
UR       2617
WS     122623
Name: TransId, dtype: int64

In [16]:
## Convert to 2020 dollars
CPI = pd.read_csv(scratchpath+'/InfAdjust.csv')
df = pd.merge(df,CPI,on='Year',how='left')
df['SalesPrice'] = df['SalesPriceAmount']/df['Deflator_base2020']
df['LoanAmt'] = df['LoanAmount']/df['Deflator_base2020']

In [17]:
## Select only valid transactions
UnwantedLoanCodes =  ['AC','CT','CM','CS','CC','CL','DP','FO','FE','HE','LC','EB','EX','MD','NA','NP','FA','RE','RM','SM','SE','TR']
WantedLoanCodes =  ['AS','BL','CE','FM','PM','RD','SL']
DocumentTypeStndCodeDrop= ['CRDE','JTDE','QCDE','RRDE','VLDE']
DocumentTypeStndCodeKeep= ['CPDE','IDDE','PRDE','WRDE']


st = df.loc[(df['SalesPriceAmount'] > 0) &
            (~df['LoanTypeStndCode'].isin(UnwantedLoanCodes)) &
#             (pd.isnull(df['IntraFamilyTransferFlag'])) &
            (df['PropertyUseStndCode']=='SR') &
            (df['DocumentTypeStndCode'].isin(DocumentTypeStndCodeKeep)) & 
            (df['TotalBedrooms'] >=1)].copy()

In [18]:
st = st.drop([
              'AssessorParcelNumber','PropertyAddressLatitude',
                 'PropertyAddressLongitude','ImportParcelID'],axis=1)

#May need to drop'TimeshareStndCode',

In [19]:
st =st.drop(['State'], axis=1)

In [20]:
rooftypes = pd.get_dummies(st['RoofCoverStndCode'],prefix='roof')

In [21]:
df = pd.concat([st, rooftypes], axis=1)

In [22]:
roofs = rooftypes.columns

In [23]:
samp = df.copy()

In [24]:
samp['Bathrooms'] = samp['FullBath'].replace(np.nan, 0) + samp['ThreeQuarterBath'].replace(np.nan, 0) + samp['HalfBath'].replace(np.nan, 0) +samp['QuarterBath'].replace(np.nan, 0)

In [25]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col
from sklearn.linear_model import LinearRegression
from scipy.stats import ttest_ind


In [26]:
roofs

Index(['roof_AP', 'roof_AS', 'roof_BR', 'roof_BU', 'roof_CN', 'roof_CS',
       'roof_FG', 'roof_GR', 'roof_ME', 'roof_OT', 'roof_RC', 'roof_SH',
       'roof_SL', 'roof_ST', 'roof_TG', 'roof_TL', 'roof_UR', 'roof_WS'],
      dtype='object')

In [27]:
roofs.values

array(['roof_AP', 'roof_AS', 'roof_BR', 'roof_BU', 'roof_CN', 'roof_CS',
       'roof_FG', 'roof_GR', 'roof_ME', 'roof_OT', 'roof_RC', 'roof_SH',
       'roof_SL', 'roof_ST', 'roof_TG', 'roof_TL', 'roof_UR', 'roof_WS'],
      dtype=object)

In [28]:
samp.groupby('RoofCoverStndCode')['TransId'].count()

RoofCoverStndCode
AP     212455
AS       1221
BR        109
BU      26141
CN       6584
CS    1860090
FG         51
GR         44
ME      27003
OT       1045
RC       2199
SH       3710
SL        912
ST          4
TG        652
TL      48841
UR       2617
WS     122623
Name: TransId, dtype: int64

In [29]:
BP_geq0 = samp.copy()

In [30]:
BP_geq0.loc[BP_geq0['RoofCoverStndCode'] =='AP', 'AsphaltRoof'] = 1
BP_geq0.loc[BP_geq0['RoofCoverStndCode'] !='AP', 'AsphaltRoof'] = 0
BP_geq0.loc[BP_geq0['RoofCoverStndCode'] =='AS', 'Asbestos'] = 1
BP_geq0.loc[BP_geq0['RoofCoverStndCode'] !='AS', 'Asbestos'] = 0
BP_geq0.loc[BP_geq0['RoofCoverStndCode'] =='BR', 'Bermuda'] = 1
BP_geq0.loc[BP_geq0['RoofCoverStndCode'] !='BR', 'Bermuda'] = 0
BP_geq0.loc[BP_geq0['RoofCoverStndCode'] =='BU', 'BuiltUpRoof'] = 1
BP_geq0.loc[BP_geq0['RoofCoverStndCode'] !='BU', 'BuiltUpRoof'] = 0
BP_geq0.loc[BP_geq0['RoofCoverStndCode'] =='CN', 'ConcreteRoof'] = 1
BP_geq0.loc[BP_geq0['RoofCoverStndCode'] !='CN', 'ConcreteRoof'] = 0
BP_geq0.loc[BP_geq0['RoofCoverStndCode'] =='CS', 'CompositShingle'] = 1
BP_geq0.loc[BP_geq0['RoofCoverStndCode'] !='CS', 'CompositShingle'] = 0
BP_geq0.loc[BP_geq0['RoofCoverStndCode'] =='ME', 'MetalRoof'] = 1
BP_geq0.loc[BP_geq0['RoofCoverStndCode'] !='ME', 'MetalRoof'] = 0
BP_geq0.loc[BP_geq0['RoofCoverStndCode'] =='TL', 'TileRoof'] = 1
BP_geq0.loc[BP_geq0['RoofCoverStndCode'] !='TL', 'TileRoof'] = 0
BP_geq0.loc[BP_geq0['RoofCoverStndCode'] =='WS', 'Wood'] = 1
BP_geq0.loc[BP_geq0['RoofCoverStndCode'] =='WG', 'Wood'] = 1
BP_geq0.loc[BP_geq0['RoofCoverStndCode'] =='WD', 'Wood'] = 1
BP_geq0.loc[BP_geq0['RoofCoverStndCode'] !='WS', 'Wood'] = 0

In [31]:
samp = BP_geq0.copy()

In [32]:
samp = samp[samp['SalesPrice'] > 10000].copy()
samp = samp[samp['SalesPrice'] < 5000000].copy()

In [33]:
samp = samp.drop_duplicates(subset=['TransId'])

In [34]:
samp['BP'].describe()

count    487429.000000
mean          0.000471
std           0.001295
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000130
max           0.011418
Name: BP, dtype: float64

In [35]:
## .001 1 in 1,000
## .0001 1 in 10,000

In [36]:
pd.options.display.float_format = '{:.2f}'.format

In [37]:
samp[samp['BP']>0].describe()

,TransId,FIPS,SalesPriceAmount,LoanAmount,LotSizeSquareFeet,GarageAreaSqFt,YearBuilt,EffectiveYearBuilt,YearRemodeled,NoOfStories,TotalRooms,TotalBedrooms,TotalKitchens,FullBath,ThreeQuarterBath,HalfBath,QuarterBath,TotalActualBathCount,TotalBathPlumbingFixtures,FireplaceNumber,LandAssessedValue,ImprovementAssessedValue,TotalAssessedValue,AssessmentYear,LandMarketValue,ImprovementMarketValue,TotalMarketValue,MarketValueYear,LandAppraisalValue,ImprovementAppraisalValue,TotalAppraisalValue,AppraisalValueYear,BP,BP1600_max,BP1600_mean,index_right,Year,StartMonth,StartDay,Acres,Distance,CIP,Deflator_base2020,SalesPrice,LoanAmt,roof_AP,roof_AS,roof_BR,roof_BU,roof_CN,roof_CS,roof_FG,roof_GR,roof_ME,roof_OT,roof_RC,roof_SH,roof_SL,roof_ST,roof_TG,roof_TL,roof_UR,roof_WS,Bathrooms,AsphaltRoof,Asbestos,Bermuda,BuiltUpRoof,ConcreteRoof,CompositShingle,MetalRoof,TileRoof,Wood
count,229815.00,229815.00,229815.00,229807.00,222733.00,956.00,229793.00,55909.00,53998.00,175967.00,229814.00,229815.00,1064.00,174150.00,1066.00,175216.00,1066.00,55664.00,48402.00,126070.00,210036.00,210200.00,229774.00,229815.00,227547.00,227714.00,229786.00,229786.00,2768.00,2768.00,2768.00,1066.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229807.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00,229815.00
mean,143814349.62,8047.00,281142.43,211779.88,118738.99,623.46,1980.25,1994.55,1994.69,1.43,4.62,3.36,1.06,2.39,0.00,0.22,0.00,2.58,6.04,1.36,7838.43,20862.90,29425.27,2017.13,110113.77,290939.28,399058.49,2017.08,61804.02,150614.14,212418.16,2017.00,0.00,0.00,0.00,4653.44,2004.21,5.24,17.39,7900.45,14001.67,191.55,0.74,374533.52,278096.60,0.06,0.00,0.00,0.00,0.01,0.52,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.07,1.98,0.06,0.00,0.00,0.00,0.01,0.52,0.02,0.02,0.07
std,103459721.92,23.57,206688.42,780128.61,2398160.63,243.93,20.63,19.45,19.54,0.52,3.73,0.93,0.25,1.01,0.00,0.54,0.00,0.78,25.81,0.62,9352.57,13342.87,22221.74,0.33,132773.44,188124.82,286745.40,0.27,122121.29,266368.38,354331.96,0.00,0.00,0.00,0.00,1516.38,6.99,2.37,7.84,16592.63,4911.40,30.79,0.12,255749.44,833780.23,0.23,0.01,0.01,0.07,0.08,0.50,0.01,0.01,0.14,0.03,0.04,0.04,0.03,0.00,0.03,0.15,0.02,0.25,1.49,0.23,0.01,0.01,0.07,0.08,0.50,0.14,0.15,0.25
min,97923044.00,8001.00,6500.00,0.00,436.00,160.00,1854.00,1885.00,1885.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,0.50,0.00,1.00,0.00,0.00,20.00,2016.00,0.00,53.00,122.00,2016.00,0.00,0.00,0.00,2017.00,0.00,0.00,0.00,4360.00,1993.00,1.00,1.00,1001.00,0.00,144.46,0.56,10890.36,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,103210173.50,8035.00,164800.00,127500.00,7398.00,484.00,1971.00,1985.00,1985.00,1.00,0.00,3.00,1.00,2.00,0.00,0.00,0.00,2.00,0.00,1.00,3240.00,13660.00,18260.00,2017.00,45000.00,191638.00,251334.00,2017.00,0.00,0.00,0.00,2017.00,0.00,0.00,0.00,4496.00,1999.00,3.00,12.00,1386.00,11396.78,166.57,0.64,235242.30,186926.90,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,104494528.00,8041.00,230000.00,184000.00,10700.00,576.00,1983.00,1999.00,1999.00,1.00,5.00,3.00,1.00,2.00,0.00,0.00,0.00,2.50,3.00,1.00,5620.00,18150.00,24520.00,2017.00,79000.00,252082.50,337911.50,2017.00,0.00,0.00,0.00,2017.00,0.00,0.00,0.00,4517.00,2003.00,5.00,20.00,2692.00,15262.08,183.96,0.71,311813.11,251643.31,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
75%,106077651.50,8059.00,335000.00,262014.00,36155.00,720.00,1996.00,2011.00,2011.00,2.00,7.00,4.00,1.00,3.00,

In [38]:
samp = samp[samp['BP']>.0001]

In [39]:
# extras    +  BP*AsphaltRoo + BP*CompositShingle + BP*MetalRoof + BP*Wood 

In [40]:
samp['BP1600_max'].describe()

count   129307.00
mean         0.00
std          0.00
min          0.00
25%          0.00
50%          0.00
75%          0.00
max          0.01
Name: BP1600_max, dtype: float64

In [41]:
samp

,TransId,FIPS,County,DataClassStndCode,RecordingDate,DocumentTypeStndCode,DocumentDate,SalesPriceAmount,PropertyUseStndCode,LoanAmount,LoanTypeStndCode,PropertyFullStreetAddress,PropertyCity,PropertyState,RowID,LotSizeSquareFeet,GarageStndCode,GarageAreaSqFt,BuildingClassStndCode,BuildingQualityStndCode,BuildingConditionStndCode,ArchitecturalStyleStndCode,YearBuilt,EffectiveYearBuilt,YearRemodeled,NoOfStories,TotalRooms,TotalBedrooms,TotalKitchens,FullBath,ThreeQuarterBath,HalfBath,QuarterBath,TotalActualBathCount,TotalBathPlumbingFixtures,RoofCoverStndCode,RoofStructureTypeStndCode,HeatingTypeorSystemStndCode,AirConditioningTypeorSystemStndCode,FoundationTypeStndCode,ElevatorStndCode,FireplaceFlag,FireplaceNumber,WaterStndCode,SewerStndCode,LandAssessedValue,ImprovementAssessedValue,TotalAssessedValue,AssessmentYear,LandMarketValue,ImprovementMarketValue,TotalMarketValue,MarketValueYear,LandAppraisalValue,ImprovementAppraisalValue,TotalAppraisalValue,AppraisalValueYear,geometry,BP,point_geometry,BP1600_max,BP1600_mean,index_right,Fire_ID,Fire_Name,Year,StartMonth,StartDay,Fire_Type,Acres,STUSPS,Buffed_geometry100km,MTBS_geometry,Distance,CIP,Deflator_base2020,SalesPrice,LoanAmt,roof_AP,roof_AS,roof_BR,roof_BU,roof_CN,roof_CS,roof_FG,roof_GR,roof_ME,roof_OT,roof_RC,roof_SH,roof_SL,roof_ST,roof_TG,roof_TL,roof_UR,roof_WS,Bathrooms,AsphaltRoof,Asbestos,Bermuda,BuiltUpRoof,ConcreteRoof,CompositShingle,MetalRoof,TileRoof,Wood
25,97925774,8001,ADAMS,H,1993-11-30,WRDE,1993-11-26,69000.00,SR,62100.00,NaN,5495 BRYANT ST,DENVER,CO,08FF4216-820C-E811-80C3-3863BB430E3F,10019.00,DT,NaN,NaN,C,AV,RA,1948.00,NaN,NaN,1.00,5.00,2.00,NaN,1.00,NaN,0.00,NaN,NaN,6.00,CS,GBL,FA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4900.00,12770.00,17670.00,2017.00,68000.00,177341.00,245341.00,2017.00,NaN,NaN,NaN,NaN,POINT (-428924.217 -565424.354),0.00,POINT (-428924.217 -565424.354),0.00,0.00,4535.00,CO3976210526320110320,INDIANGULCHFIRE,1993,3.00,20.00,Wildfire,1640.00,CO,"POLYGON ((-471930.698 -570157.810, -472068.900...","POLYGON ((-451536.725 -569134.815, -451547.548...",19832.56,144.46,0.56,123620.22,111258.20,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
40,97926641,8001,ADAMS,H,1993-12-20,WRDE,1993-12-15,110000.00,SR,104500.00,NaN,5392 QUITMAN ST,DENVER,CO,33FB6116-820C-E811-80C3-3863BB430E3F,11326.00,AT,NaN,NaN,D+,AV,RA,1947.00,NaN,NaN,1.00,7.00,4.00,NaN,2.00,NaN,0.00,NaN,NaN,9.00,CS,GBL,FA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9720.00,19680.00,29400.00,2017.00,135000.00,273362.00,408362.00,2017.00,NaN,NaN,NaN,NaN,POINT (-430680.606 -565585.132),0.00,POINT (-430680.606 -565585.132),0.00,0.00,4535.00,CO3976210526320110320,INDIANGULCHFIRE,1993,3.00,20.00,Wildfire,1640.00,CO,"POLYGON ((-471930.698 -570157.810, -472068.900...","POLYGON ((-451536.725 -569134.815, -451547.548...",18068.87,144.46,0.56,197075.71,187221.92,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
257,97937039,8001,ADAMS,H,1994-09-09,WRDE,1994-09-02,100888.00,SR,18000.00,SL,5590 CLAY ST,DENVER,CO,9C661A16-820C-E811-80C3-3863BB430E3F,9801.00,NaN,NaN,NaN,C,AV,RA,1962.00,NaN,NaN,1.00,6.00,4.00,NaN,2.00,NaN,0.00,NaN,NaN,9.00,CS,GBL,FA,CE,NaN,NaN,NaN,1.00,NaN,NaN,4900.00,15540.00,20440.00,2017.00,68000.00,215827.00,283827.00,2017.00,NaN,NaN,NaN,NaN,POINT (-428962.251 -565223.094),0.00,POINT (-428962.251 -565223.094),0.00,0.00,4535.00,CO3976210526320110320,INDIANGULCHFIRE,1994,3.00,20.00,Wildfire,1640.00,CO,"POLYGON ((-471930.698 -570157.810, -472068.900...","POLYGON ((-451536.725 -569134.815, -451547.548...",19815.45,148.22,0.57,176157.47,31429.25,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
265,97937135,8001,ADAMS,H,1994-09-13,WRDE,1994-09-06,132500.00,SR,106000.00,NaN,5431 NEWTON ST,DENVER,CO,55921516-820C-E811-80C3-3863BB430E3F,24829.00,AT,NaN,NaN,C,AV,NaN,1973.00,NaN,NaN,NaN,7.00,4.00,NaN,4.00,NaN,0.00,NaN,NaN,8.00,CS,GBL,HW,NaN,NaN,NaN,NaN,1.00,NaN,NaN,14580.00,11130.00,25710.00,2017.00,202500.

In [42]:
samp['BP'] = samp['BP']*10000

In [43]:
pd.options.display.float_format = '{:.2f}'.format

In [44]:
samp.describe()

,TransId,FIPS,SalesPriceAmount,LoanAmount,LotSizeSquareFeet,GarageAreaSqFt,YearBuilt,EffectiveYearBuilt,YearRemodeled,NoOfStories,TotalRooms,TotalBedrooms,TotalKitchens,FullBath,ThreeQuarterBath,HalfBath,QuarterBath,TotalActualBathCount,TotalBathPlumbingFixtures,FireplaceNumber,LandAssessedValue,ImprovementAssessedValue,TotalAssessedValue,AssessmentYear,LandMarketValue,ImprovementMarketValue,TotalMarketValue,MarketValueYear,LandAppraisalValue,ImprovementAppraisalValue,TotalAppraisalValue,AppraisalValueYear,BP,BP1600_max,BP1600_mean,index_right,Year,StartMonth,StartDay,Acres,Distance,CIP,Deflator_base2020,SalesPrice,LoanAmt,roof_AP,roof_AS,roof_BR,roof_BU,roof_CN,roof_CS,roof_FG,roof_GR,roof_ME,roof_OT,roof_RC,roof_SH,roof_SL,roof_ST,roof_TG,roof_TL,roof_UR,roof_WS,Bathrooms,AsphaltRoof,Asbestos,Bermuda,BuiltUpRoof,ConcreteRoof,CompositShingle,MetalRoof,TileRoof,Wood
count,129307.00,129307.00,129307.00,129303.00,125866.00,955.00,129290.00,31815.00,30346.00,100351.00,129306.00,129307.00,972.00,92088.00,974.00,93062.00,974.00,37218.00,20446.00,70410.00,121144.00,121197.00,129271.00,129307.00,127617.00,127673.00,129281.00,129281.00,2551.00,2551.00,2551.00,974.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129303.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00,129307.00
mean,146528238.38,8050.09,304578.12,228813.15,196648.17,623.73,1983.76,1994.01,1994.10,1.43,5.17,3.37,1.07,2.41,0.00,0.20,0.00,2.63,3.43,1.41,8090.45,22491.28,30570.90,2017.14,108918.95,307737.89,413880.83,2017.09,61678.28,151802.56,213480.83,2017.00,17.53,0.00,0.00,4733.60,2004.70,5.26,16.41,10420.69,13293.91,193.74,0.75,402947.36,297807.10,0.05,0.00,0.00,0.01,0.00,0.56,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.00,0.06,1.86,0.05,0.00,0.00,0.01,0.00,0.56,0.03,0.03,0.06
std,105963058.35,24.13,213276.73,1019978.12,3119872.64,243.90,20.39,19.21,19.42,0.50,3.72,0.96,0.25,0.99,0.00,0.53,0.00,0.80,13.19,0.66,9948.88,14888.20,22620.32,0.35,131651.93,190118.33,286278.60,0.29,122072.64,269480.60,357428.43,0.00,20.21,0.00,0.00,1931.68,7.02,2.06,7.66,21092.05,5422.08,30.92,0.12,265501.75,1088715.93,0.23,0.01,0.02,0.08,0.06,0.50,0.01,0.01,0.17,0.04,0.04,0.04,0.03,0.00,0.03,0.18,0.03,0.24,1.49,0.23,0.01,0.02,0.08,0.06,0.50,0.17,0.18,0.24
min,97925774.00,8001.00,6500.00,0.00,436.00,160.00,1859.00,1890.00,1890.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,0.50,0.00,1.00,0.00,0.00,20.00,2016.00,0.00,53.00,122.00,2016.00,0.00,0.00,0.00,2017.00,1.00,0.00,0.00,4360.00,1993.00,1.00,1.00,1001.00,0.00,144.46,0.56,10890.36,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,103266645.00,8041.00,176000.00,135757.50,8625.00,484.00,1974.00,1984.00,1984.00,1.00,0.00,3.00,1.00,2.00,0.00,0.00,0.00,2.25,0.00,1.00,3310.00,14230.00,18240.00,2017.00,45500.00,197526.00,251392.00,2017.00,0.00,0.00,0.00,2017.00,3.19,0.00,0.00,4496.00,1999.00,3.00,11.00,1640.00,10164.41,166.57,0.64,247421.28,194531.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,104471347.00,8041.00,253490.00,200000.00,21780.00,576.00,1987.00,1998.00,1998.00,1.00,6.00,3.00,1.00,2.00,0.00,0.00,0.00,2.50,0.00,1.00,5563.00,19600.00,25510.00,2017.00,75307.00,268901.00,350873.00,2017.00,0.00,0.00,0.00,2017.00,8.62,0.00,0.00,4514.00,2003.00,6.00,18.00,3834.00,14792.71,183.96,0.71,340483.57,269092.57,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
75%,106200431.00,8059.00,370000.00,285165.00,92347.00,720.00,1998.00,2010.00,2011.00,2.00,8.00,4.00,1.00,3.00,0.

In [45]:
samp['BP'].describe()

count   129307.00
mean        17.53
std         20.21
min          1.00
25%          3.19
50%          8.62
75%         21.83
max        114.18
Name: BP, dtype: float64

In [46]:
samp[samp['BP']>40]['BP'].mean()

57.55419796961785

In [47]:
riskdummies = pd.cut(samp.BP, bins=[0, 10, 20, 30, 40, 10000], labels = ['verylow', 
       'low', 'moderate', 'high', 'veryhigh',])

In [48]:
samp

,TransId,FIPS,County,DataClassStndCode,RecordingDate,DocumentTypeStndCode,DocumentDate,SalesPriceAmount,PropertyUseStndCode,LoanAmount,LoanTypeStndCode,PropertyFullStreetAddress,PropertyCity,PropertyState,RowID,LotSizeSquareFeet,GarageStndCode,GarageAreaSqFt,BuildingClassStndCode,BuildingQualityStndCode,BuildingConditionStndCode,ArchitecturalStyleStndCode,YearBuilt,EffectiveYearBuilt,YearRemodeled,NoOfStories,TotalRooms,TotalBedrooms,TotalKitchens,FullBath,ThreeQuarterBath,HalfBath,QuarterBath,TotalActualBathCount,TotalBathPlumbingFixtures,RoofCoverStndCode,RoofStructureTypeStndCode,HeatingTypeorSystemStndCode,AirConditioningTypeorSystemStndCode,FoundationTypeStndCode,ElevatorStndCode,FireplaceFlag,FireplaceNumber,WaterStndCode,SewerStndCode,LandAssessedValue,ImprovementAssessedValue,TotalAssessedValue,AssessmentYear,LandMarketValue,ImprovementMarketValue,TotalMarketValue,MarketValueYear,LandAppraisalValue,ImprovementAppraisalValue,TotalAppraisalValue,AppraisalValueYear,geometry,BP,point_geometry,BP1600_max,BP1600_mean,index_right,Fire_ID,Fire_Name,Year,StartMonth,StartDay,Fire_Type,Acres,STUSPS,Buffed_geometry100km,MTBS_geometry,Distance,CIP,Deflator_base2020,SalesPrice,LoanAmt,roof_AP,roof_AS,roof_BR,roof_BU,roof_CN,roof_CS,roof_FG,roof_GR,roof_ME,roof_OT,roof_RC,roof_SH,roof_SL,roof_ST,roof_TG,roof_TL,roof_UR,roof_WS,Bathrooms,AsphaltRoof,Asbestos,Bermuda,BuiltUpRoof,ConcreteRoof,CompositShingle,MetalRoof,TileRoof,Wood
25,97925774,8001,ADAMS,H,1993-11-30,WRDE,1993-11-26,69000.00,SR,62100.00,NaN,5495 BRYANT ST,DENVER,CO,08FF4216-820C-E811-80C3-3863BB430E3F,10019.00,DT,NaN,NaN,C,AV,RA,1948.00,NaN,NaN,1.00,5.00,2.00,NaN,1.00,NaN,0.00,NaN,NaN,6.00,CS,GBL,FA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4900.00,12770.00,17670.00,2017.00,68000.00,177341.00,245341.00,2017.00,NaN,NaN,NaN,NaN,POINT (-428924.217 -565424.354),1.40,POINT (-428924.217 -565424.354),0.00,0.00,4535.00,CO3976210526320110320,INDIANGULCHFIRE,1993,3.00,20.00,Wildfire,1640.00,CO,"POLYGON ((-471930.698 -570157.810, -472068.900...","POLYGON ((-451536.725 -569134.815, -451547.548...",19832.56,144.46,0.56,123620.22,111258.20,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
40,97926641,8001,ADAMS,H,1993-12-20,WRDE,1993-12-15,110000.00,SR,104500.00,NaN,5392 QUITMAN ST,DENVER,CO,33FB6116-820C-E811-80C3-3863BB430E3F,11326.00,AT,NaN,NaN,D+,AV,RA,1947.00,NaN,NaN,1.00,7.00,4.00,NaN,2.00,NaN,0.00,NaN,NaN,9.00,CS,GBL,FA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9720.00,19680.00,29400.00,2017.00,135000.00,273362.00,408362.00,2017.00,NaN,NaN,NaN,NaN,POINT (-430680.606 -565585.132),1.43,POINT (-430680.606 -565585.132),0.00,0.00,4535.00,CO3976210526320110320,INDIANGULCHFIRE,1993,3.00,20.00,Wildfire,1640.00,CO,"POLYGON ((-471930.698 -570157.810, -472068.900...","POLYGON ((-451536.725 -569134.815, -451547.548...",18068.87,144.46,0.56,197075.71,187221.92,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
257,97937039,8001,ADAMS,H,1994-09-09,WRDE,1994-09-02,100888.00,SR,18000.00,SL,5590 CLAY ST,DENVER,CO,9C661A16-820C-E811-80C3-3863BB430E3F,9801.00,NaN,NaN,NaN,C,AV,RA,1962.00,NaN,NaN,1.00,6.00,4.00,NaN,2.00,NaN,0.00,NaN,NaN,9.00,CS,GBL,FA,CE,NaN,NaN,NaN,1.00,NaN,NaN,4900.00,15540.00,20440.00,2017.00,68000.00,215827.00,283827.00,2017.00,NaN,NaN,NaN,NaN,POINT (-428962.251 -565223.094),1.70,POINT (-428962.251 -565223.094),0.00,0.00,4535.00,CO3976210526320110320,INDIANGULCHFIRE,1994,3.00,20.00,Wildfire,1640.00,CO,"POLYGON ((-471930.698 -570157.810, -472068.900...","POLYGON ((-451536.725 -569134.815, -451547.548...",19815.45,148.22,0.57,176157.47,31429.25,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
265,97937135,8001,ADAMS,H,1994-09-13,WRDE,1994-09-06,132500.00,SR,106000.00,NaN,5431 NEWTON ST,DENVER,CO,55921516-820C-E811-80C3-3863BB430E3F,24829.00,AT,NaN,NaN,C,AV,NaN,1973.00,NaN,NaN,NaN,7.00,4.00,NaN,4.00,NaN,0.00,NaN,NaN,8.00,CS,GBL,HW,NaN,NaN,NaN,NaN,1.00,NaN,NaN,14580.00,11130.00,25710.00,2017.00,202500.

In [49]:
riskdummies

25         verylow
40         verylow
257        verylow
265        verylow
329        verylow
            ...   
2825280        low
2825302        low
2825304    verylow
2825308    verylow
2825314    verylow
Name: BP, Length: 129307, dtype: category
Categories (5, object): ['verylow' < 'low' < 'moderate' < 'high' < 'veryhigh']

In [50]:
samp['risklevel'] = riskdummies

In [51]:
samp[['BuildingQualityStndCode']].value_counts()

BuildingQualityStndCode
C                          65884
B                          28748
C-                          8888
B+                          5529
B-                          3368
C+                          2658
A-                          1673
D+                          1618
D                           1391
A                            784
E                            330
A+                           131
D-                            30
dtype: int64

In [52]:
samp

,TransId,FIPS,County,DataClassStndCode,RecordingDate,DocumentTypeStndCode,DocumentDate,SalesPriceAmount,PropertyUseStndCode,LoanAmount,LoanTypeStndCode,PropertyFullStreetAddress,PropertyCity,PropertyState,RowID,LotSizeSquareFeet,GarageStndCode,GarageAreaSqFt,BuildingClassStndCode,BuildingQualityStndCode,BuildingConditionStndCode,ArchitecturalStyleStndCode,YearBuilt,EffectiveYearBuilt,YearRemodeled,NoOfStories,TotalRooms,TotalBedrooms,TotalKitchens,FullBath,ThreeQuarterBath,HalfBath,QuarterBath,TotalActualBathCount,TotalBathPlumbingFixtures,RoofCoverStndCode,RoofStructureTypeStndCode,HeatingTypeorSystemStndCode,AirConditioningTypeorSystemStndCode,FoundationTypeStndCode,ElevatorStndCode,FireplaceFlag,FireplaceNumber,WaterStndCode,SewerStndCode,LandAssessedValue,ImprovementAssessedValue,TotalAssessedValue,AssessmentYear,LandMarketValue,ImprovementMarketValue,TotalMarketValue,MarketValueYear,LandAppraisalValue,ImprovementAppraisalValue,TotalAppraisalValue,AppraisalValueYear,geometry,BP,point_geometry,BP1600_max,BP1600_mean,index_right,Fire_ID,Fire_Name,Year,StartMonth,StartDay,Fire_Type,Acres,STUSPS,Buffed_geometry100km,MTBS_geometry,Distance,CIP,Deflator_base2020,SalesPrice,LoanAmt,roof_AP,roof_AS,roof_BR,roof_BU,roof_CN,roof_CS,roof_FG,roof_GR,roof_ME,roof_OT,roof_RC,roof_SH,roof_SL,roof_ST,roof_TG,roof_TL,roof_UR,roof_WS,Bathrooms,AsphaltRoof,Asbestos,Bermuda,BuiltUpRoof,ConcreteRoof,CompositShingle,MetalRoof,TileRoof,Wood,risklevel
25,97925774,8001,ADAMS,H,1993-11-30,WRDE,1993-11-26,69000.00,SR,62100.00,NaN,5495 BRYANT ST,DENVER,CO,08FF4216-820C-E811-80C3-3863BB430E3F,10019.00,DT,NaN,NaN,C,AV,RA,1948.00,NaN,NaN,1.00,5.00,2.00,NaN,1.00,NaN,0.00,NaN,NaN,6.00,CS,GBL,FA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4900.00,12770.00,17670.00,2017.00,68000.00,177341.00,245341.00,2017.00,NaN,NaN,NaN,NaN,POINT (-428924.217 -565424.354),1.40,POINT (-428924.217 -565424.354),0.00,0.00,4535.00,CO3976210526320110320,INDIANGULCHFIRE,1993,3.00,20.00,Wildfire,1640.00,CO,"POLYGON ((-471930.698 -570157.810, -472068.900...","POLYGON ((-451536.725 -569134.815, -451547.548...",19832.56,144.46,0.56,123620.22,111258.20,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,verylow
40,97926641,8001,ADAMS,H,1993-12-20,WRDE,1993-12-15,110000.00,SR,104500.00,NaN,5392 QUITMAN ST,DENVER,CO,33FB6116-820C-E811-80C3-3863BB430E3F,11326.00,AT,NaN,NaN,D+,AV,RA,1947.00,NaN,NaN,1.00,7.00,4.00,NaN,2.00,NaN,0.00,NaN,NaN,9.00,CS,GBL,FA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9720.00,19680.00,29400.00,2017.00,135000.00,273362.00,408362.00,2017.00,NaN,NaN,NaN,NaN,POINT (-430680.606 -565585.132),1.43,POINT (-430680.606 -565585.132),0.00,0.00,4535.00,CO3976210526320110320,INDIANGULCHFIRE,1993,3.00,20.00,Wildfire,1640.00,CO,"POLYGON ((-471930.698 -570157.810, -472068.900...","POLYGON ((-451536.725 -569134.815, -451547.548...",18068.87,144.46,0.56,197075.71,187221.92,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,verylow
257,97937039,8001,ADAMS,H,1994-09-09,WRDE,1994-09-02,100888.00,SR,18000.00,SL,5590 CLAY ST,DENVER,CO,9C661A16-820C-E811-80C3-3863BB430E3F,9801.00,NaN,NaN,NaN,C,AV,RA,1962.00,NaN,NaN,1.00,6.00,4.00,NaN,2.00,NaN,0.00,NaN,NaN,9.00,CS,GBL,FA,CE,NaN,NaN,NaN,1.00,NaN,NaN,4900.00,15540.00,20440.00,2017.00,68000.00,215827.00,283827.00,2017.00,NaN,NaN,NaN,NaN,POINT (-428962.251 -565223.094),1.70,POINT (-428962.251 -565223.094),0.00,0.00,4535.00,CO3976210526320110320,INDIANGULCHFIRE,1994,3.00,20.00,Wildfire,1640.00,CO,"POLYGON ((-471930.698 -570157.810, -472068.900...","POLYGON ((-451536.725 -569134.815, -451547.548...",19815.45,148.22,0.57,176157.47,31429.25,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,verylow
265,97937135,8001,ADAMS,H,1994-09-13,WRDE,1994-09-06,132500.00,SR,106000.00,NaN,5431 NEWTON ST,DENVER,CO,55921516-820C-E811-80C3-3863BB430E3F,24829.00,AT,NaN,NaN,C,AV,NaN,1973.00,NaN,NaN,NaN,7.00,4.00,NaN,4.00,NaN,0.00,NaN,NaN,8.00,CS,GBL,HW,NaN,NaN,NaN,NaN,1.00,NaN,NaN,14580.00

In [53]:
samp['LotSizeSquareFeet'] = samp['LotSizeSquareFeet']/1000
samp['BP2'] = samp['BP']**2
samp['WoodBP'] = samp['Wood']*samp['BP']
samp['WoodBP2'] = samp['Wood']*samp['BP2']

In [ ]:
x = np.linspace(0,100,100)
y = -16.7*x**2 + 2538*x
a = np.linspace(0, 100, 100)
b = -7.9722*x**2 + 276.8*x 
fig, ax = plt.subplots()
ax.plot(x,y, label='BP')
ax.plot(a,b, label='WoodBP')
plt.legend()
plt.show()

In [54]:
## Summary Stats
samp[['SalesPrice','Year','BuildingQualityStndCode','TotalBedrooms', 'LotSizeSquareFeet',
     'Bathrooms','YearBuilt', 'BP','Wood']].describe()

,SalesPrice,Year,TotalBedrooms,LotSizeSquareFeet,Bathrooms,YearBuilt,BP,Wood
count,129307.00,129307.00,129307.00,125866.00,129307.00,129290.00,129307.00,129307.00
mean,402947.36,2004.70,3.37,196.65,1.86,1983.76,17.53,0.06
std,265501.75,7.02,0.96,3119.87,1.49,20.39,20.21,0.24
min,10890.36,1993.00,1.00,0.44,0.00,1859.00,1.00,0.00
25%,247421.28,1999.00,3.00,8.62,0.00,1974.00,3.19,0.00
50%,340483.57,2003.00,3.00,21.78,2.00,1987.00,8.62,0.00
75%,479324.32,2010.00,4.00,92.35,3.00,1998.00,21.83,0.00
max,4896559.16,2018.00,16.00,377142.48,16.00,2017.00,114.18,1.00


In [218]:
## Correlation matrix
samp[['SalesPrice','Year','BuildingQualityStndCode','TotalBedrooms', 'LotSizeSquareFeet',
     'Bathrooms','YearBuilt', 'BP', 'BP2','Wood', 'WoodBP', 'WoodBP2']].corr()

,SalesPrice,Year,TotalBedrooms,LotSizeSquareFeet,Bathrooms,YearBuilt,BP,BP2,Wood,WoodBP,WoodBP2
SalesPrice,1.00,0.09,0.25,0.01,0.35,0.08,0.13,0.12,0.08,0.05,0.04
Year,0.09,1.00,-0.02,0.02,-0.11,0.06,0.01,-0.01,-0.08,-0.04,-0.03
TotalBedrooms,0.25,-0.02,1.00,-0.01,0.15,0.22,-0.11,-0.10,0.07,0.03,0.01
LotSizeSquareFeet,0.01,0.02,-0.01,1.00,-0.01,-0.02,0.01,-0.00,-0.01,-0.00,-0.00
Bathrooms,0.35,-0.11,0.15,-0.01,1.00,0.04,0.11,0.14,0.11,0.08,0.07
YearBuilt,0.08,0.06,0.22,-0.02,0.04,1.00,-0.08,-0.10,-0.06,-0.05,-0.04
BP,0.13,0.01,-0.11,0.01,0.11,-0.08,1.00,0.97,-0.01,0.20,0.22
BP2,0.12,-0.01,-0.10,-0.00,0.14,-0.10,0.97,1.00,0.00,0.20,0.23
Wood,0.08,-0.08,0.07,-0.01,0.11,-0.06,-0.01,0.00,1.00,0.61,0.45
WoodBP,0.05,-0.04,0.03,-0.00,0.08,-0.05,0.20,0.20,0.61,1.00,0.96


In [225]:
Linear = smf.ols(formula = f'SalesPrice ~ C(Year) + BuildingQualityStndCode + TotalBedrooms  + LotSizeSquareFeet + Bathrooms +  C(YearBuilt) + Wood + BP + WoodBP', data=samp).fit()

In [255]:
samp.plot().scatter(x=samp['TotalBedrooms'], y=samp['SalesPrice'])

In [226]:
Linear = smf.ols(formula = f'SalesPrice ~  TotalBedrooms ', data=samp).fit()

In [227]:
Linear.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             SalesPrice   R-squared:                       0.064
Model:                            OLS   Adj. R-squared:                  0.064
Method:                 Least Squares   F-statistic:                     8912.
Date:                Thu, 16 Sep 2021   Prob (F-statistic):               0.00
Time:                        18:41:07   Log-Likelihood:            -1.7941e+06
No. Observations:              129307   AIC:                         3.588e+06
Df Residuals:                  129305   BIC:                         3.588e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      1.653e+05   2616.735     63.169      0.000     1.6e+05     1.7e+05
TotalBedrooms  7.053e+04    747.097     94.404      0.000    6.91e+04     7.2e+04
==============================================================================
Omnibus:                   111738.863   Durbin-Watson:                   1.556
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          5771299.023
Skew:                           3.936   Prob(JB):                         0.00
Kurtosis:                      34.768   Cond. No.                         13.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [221]:
Linear_quad = smf.ols(formula = f'SalesPrice ~ C(Year) + BuildingQualityStndCode+ TotalBedrooms  + LotSizeSquareFeet + Bathrooms +  C(YearBuilt) + Wood + BP + WoodBP + BP2 + WoodBP2', data=samp).fit()

In [222]:
Linear_quad.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             SalesPrice   R-squared:                       0.503
Model:                            OLS   Adj. R-squared:                  0.503
Method:                 Least Squares   F-statistic:                     618.0
Date:                Thu, 16 Sep 2021   Prob (F-statistic):               0.00
Time:                        11:48:06   Log-Likelihood:            -1.5986e+06
No. Observations:              117857   AIC:                         3.198e+06
Df Residuals:                  117663   BIC:                         3.200e+06
Df Model:                         193                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                      6.585e+05   1.34e+05      4.929      0.000    3.97e+05     9.2e+05
C(Year)[T.1994]                5334.3984   6805.087      0.784      0.433   -8003.465    1.87e+04
C(Year)[T.1995]                1.388e+04   6835.255      2.031      0.042     487.416    2.73e+04
C(Year)[T.1996]                1.017e+04   6763.323      1.504      0.133   -3084.275    2.34e+04
C(Year)[T.1997]                2.393e+04   6599.324      3.626      0.000     1.1e+04    3.69e+04
C(Year)[T.1998]                4.376e+04   6544.392      6.687      0.000    3.09e+04    5.66e+04
C(Year)[T.1999]                6.785e+04   6543.988     10.368      0.000     5.5e+04    8.07e+04
C(Year)[T.2000]                1.038e+05   6565.666     15.806      0.000    9.09e+04    1.17e+05
C(Year)[T.2001]                1.295e+05   6538.728     19.812      0.000    1.17e+05    1.42e+05
C(Year)[T.2002]                 1.45e+05   6575.843     22.044      0.000    1.32e+05    1.58e+05
C(Year)[T.2003]                1.508e+05   6572.347     22.944      0.000    1.38e+05    1.64e+05
C(Year)[T.2004]                1.619e+05   6621.567     24.457      0.000    1.49e+05    1.75e+05
C(Year)[T.2005]                1.776e+05   6634.872     26.763      0.000    1.65e+05    1.91e+05
C(Year)[T.2006]                1.921e+05   6729.440     28.541      0.000    1.79e+05    2.05e+05
C(Year)[T.2007]                1.932e+05   6822.930     28.322      0.000     1.8e+05    2.07e+05
C(Year)[T.2008]                1.722e+05   7241.829     23.779      0.000    1.58e+05    1.86e+05
C(Year)[T.2009]                1.328e+05   7576.154     17.527      0.000    1.18e+05    1.48e+05
C(Year)[T.2010]                 1.34e+05   7590.506     17.655      0.000    1.19e+05    1.49e+05
C(Year)[T.2011]                1.052e+05   7503.877     14.021      0.000    9.05e+04     1.2e+05
C(Year)[T.2012]                1.101e+05   7276.004     15.127      0.000    9.58e+04    1.24e+05
C(Year)[T.2013]                1.208e+05   6995.156     17.264      0.000    1.07e+05    1.34e+05
C(Year)[T.2014]                1.237e+05   6910.434     17.905      0.000     1.1e+05    1.37e+05
C(Year)[T.2015]                  1.6e+05   6848.977     23.355      0.000    1.47e+05    1.73e+05
C(Year)[T.2016]                1.762e+05   6851.854     25.720      0.000    1.63e+05     1.9e+05
C(Year)[T.2017]                1.995e+05   6806.057     29.316      0.000    1.86e+05    2.13e+05
C(Year)[T.2018]                2.114e+05   6924.910     30.524      0.000    1.98e+05    2.25e+05
BuildingQualityStndCode[T.A+]  3.984e+05   1.87e+04     21.284      0.000    3.62e+05    4.35e+05
BuildingQualityStndCode[T.A-]  -2.94e+05   8456.918    -34.770      0.000   -3.11e+05   -2.77e+05
BuildingQualityStndCode[T.B]  -7.295e+05   72

In [170]:
7.9722/(2*276.8)

0.01440065028901734

In [171]:
16.7/(2538*2)

0.003289992119779354

In [172]:
Linear_quad.f_test( 'WoodBP=WoodBP2=0')

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[3.07640279]]), p=0.0461285895475774, df_denom=1.18e+05, df_num=2>

In [189]:
Log = smf.ols(formula = f'np.log(SalesPrice) ~ C(Year) + BuildingQualityStndCode+ TotalBedrooms  + LotSizeSquareFeet + Bathrooms +  C(YearBuilt) + Wood*BP', data=samp).fit()

In [190]:
LinearLevels = smf.ols(formula = f'SalesPrice ~ C(Year) + BuildingQualityStndCode+ TotalBedrooms  + LotSizeSquareFeet + Bathrooms +  C(YearBuilt) + Wood*C(risklevel)', data=samp).fit()

In [191]:
LogLevels = smf.ols(formula = f'np.log(SalesPrice) ~ C(Year) + BuildingQualityStndCode+ TotalBedrooms  + LotSizeSquareFeet + Bathrooms + C(YearBuilt) + Wood*C(risklevel)', data=samp).fit()

In [192]:
LogSquared  = smf.ols(f'np.log(SalesPrice) ~ C(Year) + BuildingQualityStndCode+ TotalBedrooms  + LotSizeSquareFeet + Bathrooms +  C(YearBuilt) + Wood + BP + WoodBP + BP2 + WoodBP2', data=samp).fit()

In [193]:
LogSquared.f_test('WoodBP = WoodBP2 = 0')

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[10.8814996]]), p=1.8821825993840405e-05, df_denom=1.18e+05, df_num=2>

In [194]:
LogSquared.f_test('WoodBP = WoodBP2 =0')

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[10.8814996]]), p=1.8821825993840405e-05, df_denom=1.18e+05, df_num=2>

In [224]:
summary_col([Linear, Linear_quad, LinearLevels, Log, LogSquared, LogLevels], stars=True,
            regressor_order = ['Intercept','Year','BuildingQualityStndCode','TotalBedrooms', 'LotSizeSquareFeet',
     'Bathrooms','YearBuilt', 'BP', 'BP2','Wood', 'WoodBP', 'WoodBP2', 'C(risklevel)[T.verylow]', 
       'C(risklevel)[T.low]','Wood:C(risklevel)[T.low]',
                              'C(risklevel)[T.moderate]','Wood:C(risklevel)[T.moderate]',
                               'C(risklevel)[T.high]','Wood:C(risklevel)[T.high]',
                               'C(risklevel)[T.veryhigh]','Wood:C(risklevel)[T.veryhigh]'], drop_omitted=True , float_format='%.2f',)

,SalesPrice I,SalesPrice II,SalesPrice III,np.log(SalesPrice) I,np.log(SalesPrice) II,np.log(SalesPrice) III
Intercept,675659.41***,658469.66***,651914.14***,11.90***,11.86***,11.86***
,(133644.31),(133591.99),(133758.47),(0.27),(0.27),(0.28)
TotalBedrooms,29733.55***,29811.37***,29482.39***,0.08***,0.08***,0.08***
,(624.24),(623.99),(624.99),(0.00),(0.00),(0.00)
LotSizeSquareFeet,1.04***,0.99***,0.99***,0.00***,0.00***,0.00***
,(0.17),(0.17),(0.17),(0.00),(0.00),(0.00)
Bathrooms,37204.37***,37555.86***,37937.16***,0.07***,0.07***,0.07***
,(402.18),(403.34),(403.82),(0.00),(0.00),(0.00)
BP,1474.38***,2538.34***,,0.00***,0.01***,
,(28.58),(110.04),,(0.00),(0.00),


In [ ]:
samp.groupby('risklevel')['TransId'].count()

In [ ]:
Mod1.summary2()

In [ ]:
# Mod1.get_robustcov_results().summary()

In [ ]:
Mod0 = smf.ols(formula = f'SalesPrice ~ C(Year) + BuildingQualityStndCode+ TotalBedrooms + Bathrooms +  YearBuilt + BP1600_max*AsphaltRoof  + BP1600_max*CompositShingle  + BP1600_max*MetalRoof + BP1600_max*Wood', data=samp).fit()

In [ ]:
Mod0.get_robustcov_results().summary()

In [ ]:
# from stargazer.stargazer import Stargazer

In [ ]:
samp = data[['TransId','RecordingDate','SalesPriceAmount','FIPS','County','PropertyCity',
             'LotSizeSquareFeet','GarageStndCode','GarageAreaSqFt',
             'BuildingQualityStndCode','BuildingConditionStndCode','ArchitecturalStyleStndCode',
            'YearBuilt','TotalBedrooms','FullBath','ThreeQuarterBath','HalfBath','QuarterBath',
            'RoofCoverStndCode','RoofStructureTypeStndCode','HeatingTypeorSystemStndCode',
            'AirConditioningTypeorSystemStndCode','FireplaceNumber',
            'BP','BP1600_max','WoodRoof','SalesPrice','LoanAmt']].copy()